### Time for a Change Model- Abramowitz

In [1]:
import pandas as pd
import os
import numpy as np

### Vote equation
The model’s vote equation reads as:

V = A + 0.108 NETAPP + 0.543 Q2GDP + 4.313 TERM1INC

### Variable description

Static Variables for 2016

N = NETAPP	Incumbent president’s net approval rating (approval-dis-approval) in the final Gallup Poll in June

G = Q2GDP	Annualized growth rate of real GDP in the second quarter of the election year	1.2

T = TERM1INC	Presence (1) or absence (0) of a first-term incumbent in the race	0

A = CONSTANT for 2016

V = Incumbent share of the two-party presidential vote	 48.6

#### Pulling data from 1948 to 2016

In [3]:
TFC_df=pd.read_csv('TimeForAChange.csv')

In [4]:
TFC_df

,Year,Q1GDP,Q2GDP,June Net Approval,Incumbent Present,Incumbent 2+ Terms,Polarization,Incumbent Party Vote Share,Incumbent Party Win
0,2016,NaN,1.2,6.0,0,1,1,51.1,0
1,2012,2.0,1.3,-0.8,1,0,1,52.0,1
2,2008,-1.8,3.3,-30.0,0,1,-1,46.3,0
3,2004,2.7,2.6,-0.5,1,0,1,51.2,1
4,2000,1.1,8.0,19.5,0,1,1,50.3,0
5,1996,2.8,7.1,15.5,1,0,1,54.7,1
6,1992,4.5,4.3,-18.0,1,1,0,46.5,0
7,1988,2.1,5.2,10.0,0,1,0,53.9,1
8,1984,8.0,7.1,20.0,1,0,0,59.2,1
9,1980,1.3,-7.9,-21.7,1,0,0,44.7,0


In [5]:
TFC_df = TFC_df.rename({'June Net Approval':'JNA','Incumbent Present':'IP1',
                                'Incumbent 2+ Terms':'I2T', 'Polarization':'P',
                                'Incumbent Party Vote Share':'INCVS'}, axis=1)


In [6]:
TFC2008_df= TFC_df[3:18]
TFC2008_df.head()

,Year,Q1GDP,Q2GDP,JNA,IP1,I2T,P,INCVS,Incumbent Party Win
3,2004,2.7,2.6,-0.5,1,0,1,51.2,1
4,2000,1.1,8.0,19.5,0,1,1,50.3,0
5,1996,2.8,7.1,15.5,1,0,1,54.7,1
6,1992,4.5,4.3,-18.0,1,1,0,46.5,0
7,1988,2.1,5.2,10.0,0,1,0,53.9,1


In [7]:
TFC2012_df= TFC_df[2:18]
TFC2012_df.head()

,Year,Q1GDP,Q2GDP,JNA,IP1,I2T,P,INCVS,Incumbent Party Win
2,2008,-1.8,3.3,-30.0,0,1,-1,46.3,0
3,2004,2.7,2.6,-0.5,1,0,1,51.2,1
4,2000,1.1,8.0,19.5,0,1,1,50.3,0
5,1996,2.8,7.1,15.5,1,0,1,54.7,1
6,1992,4.5,4.3,-18.0,1,1,0,46.5,0


In [8]:
TFC2016_df= TFC_df[1:18]
TFC2016_df.head()

,Year,Q1GDP,Q2GDP,JNA,IP1,I2T,P,INCVS,Incumbent Party Win
1,2012,2.0,1.3,-0.8,1,0,1,52.0,1
2,2008,-1.8,3.3,-30.0,0,1,-1,46.3,0
3,2004,2.7,2.6,-0.5,1,0,1,51.2,1
4,2000,1.1,8.0,19.5,0,1,1,50.3,0
5,1996,2.8,7.1,15.5,1,0,1,54.7,1


In [9]:
TFC2020_df= TFC_df[0:18]
TFC2020_df.head()

,Year,Q1GDP,Q2GDP,JNA,IP1,I2T,P,INCVS,Incumbent Party Win
0,2016,NaN,1.2,6.0,0,1,1,51.1,0
1,2012,2.0,1.3,-0.8,1,0,1,52.0,1
2,2008,-1.8,3.3,-30.0,0,1,-1,46.3,0
3,2004,2.7,2.6,-0.5,1,0,1,51.2,1
4,2000,1.1,8.0,19.5,0,1,1,50.3,0


### Generate Regression Models

In [10]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std

## 2012 Estimates

In [15]:
y = TFC2012_df["INCVS"]
X = TFC2012_df[['Q2GDP','JNA','I2T']]
X = sm.add_constant(X)

In [16]:
model12 = sm.OLS(y, X)
results12 = model12.fit()
print(results12.summary())

                            OLS Regression Results                            
Dep. Variable:                  INCVS   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.890
Method:                 Least Squares   F-statistic:                     41.39
Date:                Mon, 21 Sep 2020   Prob (F-statistic):           1.32e-06
Time:                        08:43:10   Log-Likelihood:                -30.390
No. Observations:                  16   AIC:                             68.78
Df Residuals:                      12   BIC:                             71.87
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         51.2753      0.881     58.179      0.0

C:\Users\Chad Work Computer\anaconda\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


### 2012 Regression Estimates Match Published Equation -  Spot On
#### Incumbent vote share = 51.5 + 0.6 (Q2 growth) + 0.1 (Net approval) – 4.3 (Two+ terms). (LINZER)


In [17]:
#2012 Variables 
Q2GDP = 1.3
JNA = -.08
I2T = 0
P = 1

In [18]:
IVS = 51.5 + 0.5794*Q2GDP + 0.098*JNA -4.3201*I2T
IVS

52.24538

## 2016 Estimates

In [37]:
y = TFC2016_df["INCVS"]
X = TFC2016_df[['Q2GDP','JNA','I2T']]
X = sm.add_constant(X)

In [38]:
model16 = sm.OLS(y, X)
results16 = model16.fit()
print(results16.summary())

                            OLS Regression Results                            
Dep. Variable:                  INCVS   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.892
Method:                 Least Squares   F-statistic:                     44.84
Date:                Mon, 21 Sep 2020   Prob (F-statistic):           4.06e-07
Time:                        08:51:08   Log-Likelihood:                -31.774
No. Observations:                  17   AIC:                             71.55
Df Residuals:                      13   BIC:                             74.88
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         51.2835      0.773     66.324      0.0

C:\Users\Chad Work Computer\anaconda\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [39]:
#2016 Variables 
Q2GDP = 1.2
JNA = 6
I2T = 1
P = 1

In [40]:
### Without Polarization Variable Generted Estimates
IVS = 51.44 + 0.5799*Q2GDP + 0.098*JNA -4.2983*I2T
IVS


48.425580000000004

### 2016 1 Term Variable Equation

#### Published Estimates
http://centerforpolitics.org/crystalball/articles/forecasting-the-2016-presidential-election-will-time-for-change-mean-time-for-trump/

IVS = 47.26 + 0.543*Q2GDP + 0.108*JNA +4.313*IP1


In [27]:
y = TFC2016_df["INCVS"]
X = TFC2016_df[['Q2GDP','JNA','IP1']]
X = sm.add_constant(X)

In [28]:
model16 = sm.OLS(y, X)
results16 = model16.fit()
print(results16.summary())

                            OLS Regression Results                            
Dep. Variable:                  INCVS   R-squared:                       0.872
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     29.55
Date:                Mon, 21 Sep 2020   Prob (F-statistic):           4.48e-06
Time:                        08:43:26   Log-Likelihood:                -34.940
No. Observations:                  17   AIC:                             77.88
Df Residuals:                      13   BIC:                             81.21
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         47.1376      0.938     50.230      0.0

#### 2020 Estimates Based on Incumbent Plus 2 Terms Version of the Time for Change Model


In [46]:
y = TFC2020_df["INCVS"]
X = TFC2020_df[['Q2GDP','JNA','I2T']]
X = sm.add_constant(X)

In [47]:
model20 = sm.OLS(y, X)
results20 = model20.fit()
print(results20.summary())

                            OLS Regression Results                            
Dep. Variable:                  INCVS   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.875
Method:                 Least Squares   F-statistic:                     40.80
Date:                Mon, 21 Sep 2020   Prob (F-statistic):           3.60e-07
Time:                        08:53:21   Log-Likelihood:                -34.517
No. Observations:                  18   AIC:                             77.03
Df Residuals:                      14   BIC:                             80.60
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         51.3200      0.804     63.794      0.0

In [48]:
#2020Variables ----- Current Estimate of 2nd Quarter GDP
Q2GDP = -20 
JNA = -19
I2T = 0
P = 1

In [51]:
### Without Polarization Variable Generted Estimates
IVS = 51.32 + 0.5546*Q2GDP + 0.1094*JNA -3.99945*I2T
IVS

38.1494

In [52]:
print("Incumbend Vote Share for the 2020 presidential election is predicted to be " + str(IVS))

Incumbend Vote Share for the 2020 presidential election is predicted to be 38.1494
